# Weighting Matrix

In [1]:
import pandas as pd
import numpy as np

# Radiosensitivity rankings
sensitivity_rankings = {
    'Background': 4, 
    'Bone': 4, 
    'Obtur': 4,
    'TZ': 0,       
    'CG': 0,        
    'Bladder': 6,
    'SV': 2,        
    'Rectum': 9,   
    'NVB': 10,      
}

labels = list(sensitivity_rankings.keys())
values = list(sensitivity_rankings.values())
n_classes = len(labels)

# Create grids 
sens_of_predicted = np.array([values] * n_classes)
sens_of_actual = np.array([values] * n_classes).T

# Calculate error gap
sens_matrix = sens_of_predicted - sens_of_actual

# Apply 2x penalty for false negatives, dangerous errors (predicted < actual)
FN_weighting = 2
sens_matrix[sens_matrix < 0] = sens_matrix[sens_matrix < 0] * FN_weighting
sens_matrix = np.abs(sens_matrix)

## Calculate mean distances from radiological target

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from dataset import MedicalImageDataset, LABELS

DATA_DIR = Path("data/data")

REF_LABEL_NAMES = ["transition_zone_prostate", "central_zone_prostate"]
REF_LABEL_IDS = [LABELS[name] for name in REF_LABEL_NAMES]

# reverse the spacing for numpy (z, y, x)
SPACING_NUMPY = (2.0, 1.0, 1.0) 


def get_centroid(mask, spacing, label_ids):
    '''
    get centroid of tz/cg
    
    :param mask: strucutre masks
    :param spacing: voxel spacings
    :param label_ids: labels of tz/cg
    :return: mean position of the prostate
    '''

    # when only looking at one mask
    # voxel_coords = np.array(np.where(mask == label_id)).T
    
    # checks if mask value is in label_ids list
    is_reference = np.isin(mask, label_ids)

    # get indices of masks for each label
    voxel_coords = np.argwhere(is_reference)

    if len(voxel_coords) == 0:
        return None

    # convert to physical coords
    physical_coords = voxel_coords * np.array(spacing)
    
    # mean pos, centroid
    return np.mean(physical_coords, axis=0)

def get_avg_dist_to_point(mask, spacing, label_id, target_point):
    '''
    average distance from structure to centroid
    
    :param mask: structure masks
    :param spacing: voxel spacings
    :param label_id: structure label
    :param target_point: prostate centroid
    '''

    voxel_coords = np.array(np.where(mask == label_id)).T
    
    if len(voxel_coords) == 0:
        return None

    physical_coords = voxel_coords * np.array(spacing)
    
    # euclidean dist from all voxels to the target point
    distances = np.linalg.norm(physical_coords - target_point, axis=1)
    
    return np.mean(distances)

def main():
    # dataset
    data = sorted(DATA_DIR.glob("*_img.nii*"))
    images = []
    masks = []

    for img in data:
        # seperate the masks and image files
        mask_file = img.parent / img.name.replace('_img', '_mask')
        if mask_file.exists():
            images.append(img)
            masks.append(mask_file)

    dataset = MedicalImageDataset(images, masks, augment=False)
    
    # results in dict
    label_distances = {name: [] for name in LABELS.keys()}

    for i in tqdm(range(len(dataset))): # tqdm speeds up loops
        _, mask_tensor, _ = dataset[i]
        mask = mask_tensor.numpy()
        
        # centre of central zone 
        centroid = get_centroid(mask, SPACING_NUMPY, REF_LABEL_IDS)

        # distance from every class to central zone
        for name, label_id in LABELS.items():
            dist = get_avg_dist_to_point(mask, SPACING_NUMPY, label_id, centroid)
            
            if dist is not None:
                label_distances[name].append(dist)

    # output stats
    print("\nAverage Distance to Prostate Centroid (mm):")
    print(f"{'Structure':<25} | {'Mean':<5} | {'Std':<5}")
    print("-" * 50)
    
    for name, distances in label_distances.items():            
        arr = np.array(distances)
        print(f"{name:<25} | {np.mean(arr):.2f} | {np.std(arr):.2f}")

    return label_distances

label_distances = main()

In [ ]:
# Placeholder of distance matrix:

distances = list(label_distances.values())

distance_matrix = np.ones((9, 9))
for i in range(9):
    distance_matrix[i, :] = distances[i]

In [ ]:
pd.DataFrame(distance_matrix.round(2), index=labels, columns=labels)

array([[81.75, 81.75, 81.75, 81.75, 81.75, 81.75, 81.75, 81.75, 81.75],
       [20.27, 20.27, 20.27, 20.27, 20.27, 20.27, 20.27, 20.27, 20.27],
       [14.44, 14.44, 14.44, 14.44, 14.44, 14.44, 14.44, 14.44, 14.44],
       [45.27, 45.27, 45.27, 45.27, 45.27, 45.27, 45.27, 45.27, 45.27],
       [49.67, 49.67, 49.67, 49.67, 49.67, 49.67, 49.67, 49.67, 49.67],
       [81.98, 81.98, 81.98, 81.98, 81.98, 81.98, 81.98, 81.98, 81.98],
       [47.8 , 47.8 , 47.8 , 47.8 , 47.8 , 47.8 , 47.8 , 47.8 , 47.8 ],
       [38.29, 38.29, 38.29, 38.29, 38.29, 38.29, 38.29, 38.29, 38.29],
       [29.5 , 29.5 , 29.5 , 29.5 , 29.5 , 29.5 , 29.5 , 29.5 , 29.5 ]])

In [104]:
distance_weighting = 1 / distance_matrix

distance_matrix = (distance_weighting * 81 / distance_weighting.sum())

# Normalize
pd.DataFrame(distance_matrix.round(2), index=labels, columns=labels)

,Background,Bone,Obtur,TZ,CG,Bladder,SV,Rectum,NVB
Background,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41
Bone,1.67,1.67,1.67,1.67,1.67,1.67,1.67,1.67,1.67
Obtur,2.34,2.34,2.34,2.34,2.34,2.34,2.34,2.34,2.34
TZ,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75
CG,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68
Bladder,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41
SV,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71
Rectum,0.88,0.88,0.88,0.88,0.88,0.88,0.88,0.88,0.88
NVB,1.15,1.15,1.15,1.15,1.15,1.15,1.15,1.15,1.15


In [ ]:
accuracy_matrix = 1 - np.eye(9) 
# Used for asserting that we want segmentations to be accurate in general, to give doctors a better picture of the overall image
# Equivalent to regular dice loss

In [106]:
print(pd.DataFrame(sens_matrix.round(1), index=labels, columns=labels))
print('-' * 50)
print(pd.DataFrame(distance_matrix.round(1), index=labels, columns=labels))
print('-' * 50)
print(pd.DataFrame(accuracy_matrix.round(1), index=labels, columns=labels))

            Background  Bone  Obtur  TZ  CG  Bladder  SV  Rectum  NVB
Background           0     0      0   8   8        2   4       5    6
Bone                 0     0      0   8   8        2   4       5    6
Obtur                0     0      0   8   8        2   4       5    6
TZ                   4     4      4   0   0        6   2       9   10
CG                   4     4      4   0   0        6   2       9   10
Bladder              4     4      4  12  12        0   8       3    4
SV                   2     2      2   4   4        4   0       7    8
Rectum              10    10     10  18  18        6  14       0    1
NVB                 12    12     12  20  20        8  16       2    0
--------------------------------------------------
            Background  Bone  Obtur   TZ   CG  Bladder   SV  Rectum  NVB
Background         0.4   0.4    0.4  0.4  0.4      0.4  0.4     0.4  0.4
Bone               1.7   1.7    1.7  1.7  1.7      1.7  1.7     1.7  1.7
Obtur              2.3   2.3  

In [ ]:
# Final matrix calculation:
sens_matrix = sens_matrix * distance_matrix
# Normalize
sens_matrix = sens_matrix * 72 / sens_matrix.sum() # Now both sens and acc matrices have a total of 72

sens_importance = 0.9 # We say that sensitivity distinguishing is twice as important as regular dice loss

# Calculate overall importance
final_matrix = (sens_importance * sens_matrix + (1 - sens_importance) * accuracy_matrix)

df = pd.DataFrame(np.round(final_matrix, 2), index=labels, columns=labels)

df

,Background,Bone,Obtur,TZ,CG,Bladder,SV,Rectum,NVB
Background,0.00,0.10,0.10,0.59,0.59,0.22,0.34,0.40,0.47
Bone,0.10,0.00,0.10,2.07,2.07,0.59,1.08,1.33,1.57
Obtur,0.10,0.10,0.00,2.86,2.86,0.79,1.48,1.83,2.17
TZ,0.54,0.54,0.54,0.00,0.10,0.76,0.32,1.09,1.20
CG,0.50,0.50,0.50,0.10,0.00,0.70,0.30,1.00,1.10
Bladder,0.34,0.34,0.34,0.83,0.83,0.00,0.59,0.28,0.34
SV,0.31,0.31,0.31,0.52,0.52,0.52,0.00,0.83,0.93
Rectum,1.40,1.40,1.40,2.44,2.44,0.88,1.92,0.00,0.23
NVB,2.13,2.13,2.13,3.48,3.48,1.45,2.80,0.44,0.00


In [98]:
# Check that it's normalized, should be 72
df.sum().sum()

np.float64(71.98)

# Metrics and functions

### Discrete metric function

In [200]:
import torch

def compute_weighted_dice_score(pred, target, weight_matrix, num_classes=3, epsilon=1e-6):

    pred = pred.view(-1).long()
    target = target.view(-1).long()

    ids = target * num_classes + pred
    
    cm = torch.bincount(ids, minlength=num_classes**2).view(num_classes, num_classes).t().float()

    weighted_cm = cm * weight_matrix

    tp = cm.diagonal()

    fp = weighted_cm.sum(dim=1) # Summing across rows, where we always predict a particular class

    fn = weighted_cm.sum(dim=0) # Summing across columns, where a particular class is always the target

    score = (2 * tp) / (2 * tp + fp + fn + epsilon)

    return score

## Testing

In [201]:
weight_matrix = final_matrix

target = torch.Tensor([[
    [8, 1, 1, 3],
    [5, 1, 1, 6],
    [1, 1, 1, 0],
    [2, 2, 0, 0],
]])

pred = torch.Tensor([[
    [8, 1, 1, 8],
    [5, 1, 1, 5],
    [2, 1, 1, 1],
    [2, 2, 0, 0],
]])


scores = compute_weighted_dice_score(pred, target, weight_matrix, num_classes=9)

print("Confusion Matrix (Internal Calculation):")

print("-" * 30)
for i in range(9):
    print(f"Class {i} Loss: {scores[i]:.4f}")
print("-" * 30)
print(f"Mean Dice Loss: {scores.mean():.4f}")

Confusion Matrix (Internal Calculation):
------------------------------
Class 0 Loss: 0.8763
Class 1 Loss: 0.8984
Class 2 Loss: 0.8346
Class 3 Loss: 0.0000
Class 4 Loss: 0.0000
Class 5 Loss: 0.8010
Class 6 Loss: 0.0000
Class 7 Loss: 0.0000
Class 8 Loss: 0.4106
------------------------------
Mean Dice Loss: 0.4245


C:\Users\timcy\AppData\Local\Temp\ipykernel_14348\3752216378.py:12: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  weighted_cm = cm * weight_matrix


## Differentiable loss function

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# This assumes that the target is input not already one-hotted
class WeightedDiceScore(nn.Module):
    def __init__(self, weight_matrix, epsilon=1e-6):
        super().__init__()
        # self.register_buffer("weight_matrix", weight_matrix)
        self.weight_matrix = weight_matrix
        self.epsilon = epsilon

    def forward(self, pred, target):
        """
        Args:
            pred (torch.Tensor): Probabilities (B, C, H, W) - Output of Softmax
            target (torch.Tensor): Ground Truth Indices (B, H, W)
        """
        num_classes = pred.shape[1]
        
        # 1. Convert Target to One-Hot (B, C, H, W)
        # We assume target has shape (B, H, W)
        target_onehot = F.one_hot(target, num_classes).permute(0, 3, 1, 2).float()

        # 2. Flatten spatial dimensions for efficient matrix multiplication
        # Shapes become: (B, C, N) where N = H*W
        pred_flat = pred.flatten(2) 
        target_flat = target_onehot.flatten(2)
        
        # 3. Compute Soft Confusion Matrix via Einsum
        # Equation: For every batch b, sum over pixels n: pred(c) * target(k)
        # Result: (C, C) matrix where row=Pred, col=Target
        # This effectively sums up the "probability mass" for every pred/target pair
        soft_cm = torch.einsum("bcn, bkn -> ck", pred_flat, target_flat)
        
        # 4. Apply Weights
        # weight_matrix is (C, C). Element-wise multiplication applies penalties.
        # Since diagonal of weight_matrix is 0, TP contributions are zeroed out here.
        weighted_cm = soft_cm * self.weight_matrix

        # 5. Calculate Components
        
        # TP: Diagonal of the original Soft Confusion Matrix (unweighted)
        tp = torch.diagonal(soft_cm)
        
        # FP_weighted: Sum of Weighted Rows (Predicted c, Actual k)
        # sum(dim=1) collapses columns
        fp_weighted = weighted_cm.sum(dim=1)
        
        # FN_weighted: Sum of Weighted Columns (Actual c, Predicted k)
        # sum(dim=0) collapses rows
        fn_weighted = weighted_cm.sum(dim=0)

        # 6. Dice Formula
        numerator = 2 * tp
        denominator = (2 * tp) + fp_weighted + fn_weighted + self.epsilon
        
        scores = numerator / denominator
        
        return scores

## Testing

In [203]:
weight_matrix = torch.Tensor([
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 0],
])

# Create a differentiable "Model Output" (Logits)
# Batch=1, Classes=3, Height=2, Width=2
# logits = torch.randn(1, 3, 2, 2, requires_grad=True)
a = 1000.0
logits = torch.tensor([[
    [[a, 0.0],
    [0.0, 0.0]],
    [[0.0, a],
    [-100, 0.0]],
    [[0.0, 0.0],
    [0.0, a]],
]], requires_grad=True)

logits2 = torch.zeros((1, 6, 2, 2))

logits = torch.cat([logits, logits2], dim=1)

# Apply Softmax to get probabilities (Required for this loss)
probs = F.softmax(logits, dim=1)

print(logits.shape)
print(probs.shape)

# Target (Indices)
target = torch.tensor([[[0, 1],
                        [1, 2]]]) # Shape (1, 2, 2)

# 2. Initialize Module
dice_calc = WeightedDiceScore(torch.Tensor(final_matrix))

# 3. Forward Pass
scores = dice_calc(probs, target)
print(f"Dice Scores per class: {scores}")

torch.Size([1, 9, 2, 2])
torch.Size([1, 9, 2, 2])
Dice Scores per class: tensor([0.9913, 0.7313, 0.9528, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
       grad_fn=<DivBackward0>)
